In [1]:
# In[1]: Imports and global settings

import numpy as np
import random
from collections import defaultdict
import time

# Grid & agent parameters
GRID_SIZE   = 5
NUM_AGENTS  = 4
A_POS       = (0, 0)        # Pickup cell
B_POS       = (4, 4)        # Drop-off cell
ACTIONS     = [(-1,0),(1,0),(0,-1),(0,1)]  # N, S, W, E
NUM_ACTIONS = len(ACTIONS)

# Q-learning hyperparameters
ALPHA   = 0.1   # learning rate
GAMMA   = 0.99  # discount factor
EPSILON = 0.2   # exploration rate

# Training budgets (step, collision, time)
STEP_BUDGET   = 1_500_000
COLLISION_BUDGET = 4_000
WALLTIME_BUDGET = 600    # seconds

# Performance test parameters
TEST_EPISODES     = 1000
TEST_STEP_LIMIT   = 20
FINAL_SUCCESS_RATE = 0.75


In [5]:
# In[ ]: Updated Environment with off-the-job flag

class MultiAgentEnv:
    def __init__(self, off_job_training: bool = True):
        """
        off_job_training=True:
          - fixed start: 2 agents at A, 2 at B each episode
        off_job_training=False:
          - random start: all agents and A/B randomised each episode
        """
        self.size      = GRID_SIZE
        self.num_agents = NUM_AGENTS
        self.off_job   = off_job_training
        self.reset()

    def reset(self):
        if self.off_job:
            # ---- Off‐the‐job training: fixed 2 at A, 2 at B ----
            self.agent_pos = [A_POS]*2 + [B_POS]*2
            self.carrying   = [False]*2 + [True]*2
        else:
            # ---- Standard: randomise agent starts and initial carrying states ----
            self.agent_pos = []
            self.carrying  = []
            for _ in range(self.num_agents):
                # random between A or B
                if random.random() < 0.5:
                    self.agent_pos.append(A_POS)
                    self.carrying.append(False)
                else:
                    self.agent_pos.append(B_POS)
                    self.carrying.append(True)

        self.steps      = 0
        self.collisions = 0
        return self._get_observations()

    def _get_observations(self):
        """
        Opposite‐direction sensor ONLY:
        for each of the 8 neighbours, set 1 if occupied by an agent
        whose direction (A→B vs B→A) is opposite to self.
        """
        obs = []
        for i in range(self.num_agents):
            x, y    = self.agent_pos[i]
            dir_i   = 'A2B' if not self.carrying[i] else 'B2A'
            opp_mask = []
            for dx in (-1, 0, 1):
                for dy in (-1, 0, 1):
                    if dx == 0 and dy == 0:
                        continue
                    nx, ny = x+dx, y+dy
                    bit = 0
                    if 0 <= nx < self.size and 0 <= ny < self.size:
                        for j in range(self.num_agents):
                            if (nx, ny) == self.agent_pos[j]:
                                dir_j = 'A2B' if not self.carrying[j] else 'B2A'
                                if dir_j != dir_i:
                                    bit = 1
                                break
                    opp_mask.append(bit)
            obs.append((x, y, int(self.carrying[i]), tuple(opp_mask)))
        return obs

    def step(self, actions):
        """
        Central clock update in fixed order 0→1→…→N−1.
        Count only head‐on collisions.
        """
        self.steps += 1
        prev = list(self.agent_pos)
        for i, a in enumerate(actions):
            dx, dy = ACTIONS[a]
            x, y   = self.agent_pos[i]
            nx, ny = x+dx, y+dy
            # wall check
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.agent_pos[i] = (nx, ny)
            # detect head-on with any earlier-updated agent
            for j in range(i):
                if (self.agent_pos[i] == prev[j] and
                    self.agent_pos[j] == prev[i]):
                    self.collisions += 1
            # auto pickup/dropoff
            if not self.carrying[i] and self.agent_pos[i] == A_POS:
                self.carrying[i] = True
            elif self.carrying[i] and self.agent_pos[i] == B_POS:
                self.carrying[i] = False

        return self._get_observations(), self.collisions


In [6]:
# In[3]: Independent Tabular Q-learning agents

class QAgent:
    def __init__(self):
        # Q-table mapping state→action values
        self.Q = defaultdict(lambda: np.zeros(NUM_ACTIONS))

    def select_action(self, state):
        # epsilon-greedy
        if random.random() < EPSILON:
            return random.randrange(NUM_ACTIONS)
        qvals = self.Q[state]
        return int(np.argmax(qvals))

    def update(self, state, action, reward, next_state):
        q = self.Q[state][action]
        q_next = np.max(self.Q[next_state])
        # Q-learning update
        self.Q[state][action] = q + ALPHA*(reward + GAMMA*q_next - q)


In [76]:
# In[4]: Non-episodic Q-learning training loop 
import time
# 1) Test with just Central clock + Opposite-direction sensor:
# 2) Then, if you want to add Off-the-job training as well:
# env = MultiAgentEnv(off_job_training=True)
# In[4]: Training loop (fixed)

# Instantiate with or without off-the-job as you like:
env = MultiAgentEnv(off_job_training=False)

# Create one Q-learning agent per grid agent
agents = [QAgent() for _ in range(NUM_AGENTS)]

# Initialize counters and get initial (non-episodic) state
total_steps      = 0
total_collisions = 0
start_time       = time.time()
states           = env.reset()

# Continue learning until any budget is hit
while (total_steps < STEP_BUDGET
       and total_collisions < COLLISION_BUDGET
       and time.time() - start_time < WALLTIME_BUDGET):

    # 1) Central‐clock action selection
    actions = [agents[i].select_action(states[i])
               for i in range(NUM_AGENTS)]

    # 2) Environment step: returns next states and cumulative collisions
    prev_collisions = total_collisions
    next_states, collisions = env.step(actions)

    # 3) Q-learning updates for each agent
    for i in range(NUM_AGENTS):
        reward = -1                       # step penalty

        # collision penalty if any new head-on collision occurred
        if collisions > prev_collisions:
            reward -= 10

        # successful drop-off detection (A→B→A cycle)
        # we know a drop-off just happened if carrying→not carrying
        if states[i][2] == 1 and next_states[i][2] == 0:
            reward += 10

        # update Q-table for agent i
        agents[i].update(states[i], actions[i], reward, next_states[i])

    # 4) Advance for next loop iteration
    states = next_states
    total_steps      += NUM_AGENTS
    total_collisions  = collisions

# summary
print(f"Training finished in {total_steps} steps, "
      f"{total_collisions} collisions, "
      f"time elapsed {int(time.time()-start_time)} s")


Training finished in 308540 steps, 4000 collisions, time elapsed 4 s


In [75]:
# hyperparameters
ALPHA             = 0.01    # lower LR
GAMMA             = 0.99
COLLISION_PENALTY = 100     # heavy penalty
EPS_START, EPS_END = 1.0, 0.01
TOTAL_AGENT_ACTIONS = STEP_BUDGET  # 1.5M
eps_decay         = (EPS_START - EPS_END)/(TOTAL_AGENT_ACTIONS/2)

# instantiate
env     = MultiAgentEnv(off_job_training=True)
agents  = [QAgent() for _ in range(NUM_AGENTS)]
states  = env.reset()
epsilon = EPS_START
steps   = 0
colls   = 0
start   = time.time()

while (steps < STEP_BUDGET 
       and colls < COLLISION_BUDGET 
       and time.time()-start < WALLTIME_BUDGET):

    prev_colls = env.collisions
    actions    = []
    # select with linear decay epsilon
    for i in range(NUM_AGENTS):
        if random.random() < epsilon:
            actions.append(random.randrange(NUM_ACTIONS))
        else:
            actions.append(int(np.argmax(agents[i].Q[states[i]])))

    # decay eps
    epsilon = max(EPS_END, epsilon - eps_decay)

    next_states, _ = env.step(actions)
    new_colls      = env.collisions
    n_coll         = new_colls - prev_colls

    # Q-updates
    for i in range(NUM_AGENTS):
        r = -1  # step cost

        # neighbourhood anticipation penalty
        # unpack the last 8 bits of the state
        neigh_bits = states[i][3]
        if any(neigh_bits):
            r -= 2

        # collision penalty only if this agent actually collided?
        # (for now we apply to all if any head-on happened)
        if n_coll > 0:
            r -= COLLISION_PENALTY * n_coll

        # successful dropoff
        if states[i][2]==1 and next_states[i][2]==0:
            r += 10

        agents[i].update(states[i], actions[i], r, next_states[i])

    states = next_states
    steps  += NUM_AGENTS
    colls   = new_colls

print(f"Done: {steps} steps, {colls} collisions")


Done: 246948 steps, 4000 collisions


In [70]:
# In[ ]: Performance evaluation (Table 2)

# 1) Test success rate (≤20 steps, no collisions)
test_env = MultiAgentEnv(off_job_training=False)
successes = 0

for _ in range(TEST_EPISODES):
    states   = test_env.reset()
    collided = False

    for t in range(TEST_STEP_LIMIT):
        actions, prev_coll = [], test_env.collisions
        for i in range(NUM_AGENTS):
            # greedy (no epsilon) action
            actions.append(agents[i].select_action(states[i]))
        next_states, _ = test_env.step(actions)

        # detect any collision in this test step
        if test_env.collisions > prev_coll:
            collided = True
        # detect dropoff completion (A→B→A cycle)
        if not collided and any(
                states[i][2] == 1 and next_states[i][2] == 0
                for i in range(NUM_AGENTS)
        ):
            successes += 1
            break

        states = next_states

success_rate = successes / TEST_EPISODES * 100

# 2) Training collisions (head-on) already tracked in `total_collisions`
train_collisions = total_collisions

print(f"Test success rate (≤{TEST_STEP_LIMIT} steps): {success_rate:.1f}%")
print(f"Head-on collisions during training: {train_collisions}")

# 3) Compute Performance Points per Table 2
perf_points = 0
if success_rate > 95 and train_collisions < 500:
    perf_points = 2
elif success_rate > 85 and train_collisions < 1000:
    perf_points = 1

print(f"Performance points earned: {perf_points} / 2 per metric, 4 max")


Test success rate (≤20 steps): 48.4%
Head-on collisions during training: 16443
Performance points earned: 0 / 2 per metric, 4 max


In [48]:
# In[ ]: Performance evaluation (Table 2 measurement)

# Use the same environment configuration as training (toggle off_job_training if needed)
test_env = MultiAgentEnv(off_job_training=False)

successes = 0
total_test_collisions = 0

for _ in range(TEST_EPISODES):
    states = test_env.reset()
    prev_coll = 0

    # Run up to TEST_STEP_LIMIT steps to attempt one A→B→A round-trip
    for t in range(TEST_STEP_LIMIT):
        # Select each agent’s greedy action (no exploration)
        actions = [agents[i].select_action(states[i]) for i in range(NUM_AGENTS)]
        next_states, coll = test_env.step(actions)

        # Count only new head-on collisions in this step
        total_test_collisions += (coll - prev_coll)
        prev_coll = coll

        # Detect successful drop-off: carrying→not carrying
        if any(states[i][2] == 1 and next_states[i][2] == 0
               for i in range(NUM_AGENTS)):
            successes += 1
            break

        states = next_states

# Compute metrics
success_rate = successes / TEST_EPISODES * 100
print(f"Success rate (≤ {TEST_STEP_LIMIT} steps, collision-free): {success_rate:.1f}%")
print(f"Total head-on collisions over {TEST_EPISODES} tests: {total_test_collisions}")


Success rate (≤ 20 steps, collision-free): 100.0%
Total head-on collisions over 1000 tests: 244


In [ ]:
# In[6]: Final performance check

assert rate >= FINAL_SUCCESS_RATE, \
       f"Final rate {rate:.2f} below target {FINAL_SUCCESS_RATE:.2f}"
print("Meets final delivery requirement.")
